In [9]:
import os
import os.path as osp
from pathlib import Path
import json
import pickle
import collections as C
import itertools as I

from common.constants import CORE_OPTIONS
from common.utils import remove_comments, remove_spaces
from common.pantograph.dataclasses import ProblemGenerationProcess

header = ("""
import Mathlib
import Aesop

""" + '\n'.join('set_option ' + t.replace('=', ' ') for t in CORE_OPTIONS)).strip()
print(header)

import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option maxRecDepth 100000
set_option tactic.hygienic false
set_option pp.fullNames true
set_option pp.funBinderTypes true
set_option pp.piBinderTypes true


In [26]:
# result_pkl_path = '/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean.problem_generator/problem_generation.20250822-005458.pkl'
# result_pkl_path = '/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/problem_generation.20250828-223616.pkl'
# result_pkl_path = '/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack/problem_generation.20250903-221358.pkl'
result_pkl_path = '/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_wg/Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack/problem_generation.20250828-210118.pkl'

In [27]:
path_chunks = result_pkl_path.strip('/').split('/')
agent = path_chunks[-3]
model_name = path_chunks[-2]
print(agent, model_name)

sft_wg Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack


In [28]:
with open(result_pkl_path, 'rb') as f:
    data = pickle.load(f)

results_all = C.defaultdict(list)
idx_cnt = C.Counter()
for ((ptype, source), idx), v in data.items():
    idx_cnt[idx] += 1
    results_all[(ptype, source)].append(v)

In [29]:
KC_cnt = C.Counter()
for v in results_all.values():
    for vv in v:
        if (vv.formal_solution_draft or '') != '':
            KC_cnt[len(remove_spaces(remove_comments(vv.formal_solution_draft)))] += 1
print(sum((k * v) for (k, v) in KC_cnt.items()) / sum(KC_cnt.values()))

250.19268635724333


In [ ]:
print('''
Agent\t{}
Model\t{}
#Proven\t{}
#Typechecked\t{}
#All\t{}
#Prompt Tokens\t{}
#Completion Tokens\t{}
Avg. KC@1\t{}
'''.format(
agent,
model_name,
sum([len([vv for vv in v if (vv.formal_solution_draft or '') != '']) for v in results_all.values()]),
sum([len([vv for vv in v if (vv.formal_statement or '') != '']) for v in results_all.values()]),
sum([len(v) for v in results_all.values()]),
sum([json.loads(vv.metainfo).get('prompt_tokens', 0) for v in results_all.values() for vv in v]),
sum([json.loads(vv.metainfo).get('prompt_tokens', 0) for v in results_all.values() for vv in v]),
sum((k * v) for (k, v) in KC_cnt.items()) / sum(KC_cnt.values())
))



Agent	sft_wg
Model	Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack
#Proven	711
#Typechecked	3173
#All	3800
#Prompt Tokens	2230764
#Completion Tokens	2230764
Avg. KC	250.19268635724333



In [ ]:
for (ptype, source), ds in results_all.items():
    output_lean_root = osp.join('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/results', model_name)
    os.makedirs(output_lean_root, exist_ok=True)
    with open(osp.join(output_lean_root, f'{ptype}.{source}.lean'), 'w') as f:
        f.write(header + '\n\n')
        for result in ds:
            if result.formal_solution_draft is not None:
                assert result.formal_statement.endswith(':= sorry')
                f.write('\n'.join(l + ' in' for l in result.header.splitlines()) + '\n' + result.formal_statement[:-len(' sorry')] + ' by\n' + result.formal_solution_draft + '\n\n')